# Hailo HEF Export (Raspberry PI AI Hat 13 TOPS)

This export notebook it to help with converting a yolo pytorch model to .hef format for use on a raspberry pi ai hat 13 tops model.

Under the hood this is a hailo-8l architecture.

There are 2 examples, one for a yolo object detection model, one for a segmentation model.

Machine setup:

This notebook has been tested on a 4060 TI 16GB x86 machine with 32 Gb RAM running Ubuntu 22.04

This notebook does *NOT* run in google colab free tier, it might work on some of the larger paid hardware options however.

Start by copying your pytorch weights from the yolo training process.

We will also pull the dataset from oslo.vision and use that for the calibration images. If you are not using oslo just upload a folder of images.


In [ ]:
# install hailo deps

!wget https://s3.amazonaws.com/excitedmedia/hailo_dataflow_compiler-3.32.0-py3-none-linux_x86_64.whl
!git clone https://github.com/hailo-ai/hailo_model_zoo.git
!sudo apt update && sudo apt install -y graphviz graphviz-dev
!pip install hailo_dataflow_compiler-3.32.0-py3-none-linux_x86_64.whl
!cd hailo_model_zoo && git checkout 4e9e33d1c432087723223e669e33eca3a8f9a14c
!cd hailo_model_zoo && pip install -e .



--2025-09-16 09:05:06--  https://s3.amazonaws.com/excitedmedia/hailo_dataflow_compiler-3.32.0-py3-none-linux_x86_64.whl
Resolving s3.amazonaws.com (s3.amazonaws.com)... 3.5.24.77, 52.217.229.48, 52.216.217.88, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|3.5.24.77|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 512214880 (488M) [binary/octet-stream]
Saving to: ‘hailo_dataflow_compiler-3.32.0-py3-none-linux_x86_64.whl.1’

hailo_dataflow_comp 100%[===================>] 488.49M  26.7MB/s    in 19s     

2025-09-16 09:05:25 (26.2 MB/s) - ‘hailo_dataflow_compiler-3.32.0-py3-none-linux_x86_64.whl.1’ saved [512214880/512214880]

Cloning into 'hailo_model_zoo'...
remote: Enumerating objects: 10755, done.
remote: Counting objects: 100% (2289/2289), done.
remote: Compressing objects: 100% (875/875), done.
remote: Total 10755 (delta 2066), reused 1416 (delta 1414), pack-reused 8466 (from 4)
Receiving objects: 100% (10755/10755), 6.77 MiB | 19.91 MiB/s, done.
Res

In [10]:
!pip install -U ultralytics

import ultralytics as ul
ul.checks()

import os

HOME = os.getcwd()
print("HOME:", HOME)

Ultralytics 8.3.200 🚀 Python-3.10.12 torch-2.8.0+cu128 CUDA:0 (NVIDIA GeForce RTX 4060 Ti, 15950MiB)
Setup complete ✅ (24 CPUs, 30.5 GB RAM, 537.6/767.9 GB disk)
HOME: /home/pyrat/projects/training-notebooks/notebooks


In [ ]:
# pull Dataset from Oslo for use as calibration images in the convertion process.
# This assumes that you have your dataset in oslo.vision and have created an export in the correct format.
# (YOLO for Object Detection, YOLO-Seg for segmentation)

!pip install -U oslovision

from oslovision import OsloVision

# Replace these with your own values

project_identifier = "[YOUR_PROJECT_IDENTIFIER]"
access_token = "[YOUR_ACCESS_TOKEN]"
dataset_version = "[YOUR_DATASET_VERSION]"

oslo = OsloVision(access_token)

# Download dataset
dataset_path = oslo.download_export(project_identifier, dataset_version, HOME)

print("Dataset path:", dataset_path)

  Using cached oslovision-0.1.7-py3-none-any.whl (6.0 kB)
Dataset path: /home/pyrat/projects/training-notebooks/notebooks/number-plates-yolo-seg-2


In [12]:
# export to onnx format

!yolo export model={HOME}/../best.pt imgsz=640 format=onnx opset=11

Ultralytics 8.3.200 🚀 Python-3.10.12 torch-2.8.0+cu128 CPU (AMD Ryzen 9 7900 12-Core Processor)
YOLOv8s-seg summary (fused): 85 layers, 11,779,987 parameters, 0 gradients, 39.9 GFLOPs

PyTorch: starting from '/home/pyrat/projects/training-notebooks/notebooks/../best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) ((1, 37, 8400), (1, 32, 160, 160)) (22.7 MB)
requirements: Ultralytics requirement ['onnxslim>=0.1.67'] not found, attempting AutoUpdate...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.7/164.7 KB 4.4 MB/s eta 0:00:00

requirements: AutoUpdate success ✅ 1.3s
WARNING ⚠️ requirements: Restart runtime or rerun command for updates to take effect


ONNX: starting export with onnx 1.16.0 opset 11...
ONNX: slimming with onnxslim 0.1.68...
ONNX: export success ✅ 2.2s, saved as '/home/pyrat/projects/training-notebooks/notebooks/../best.onnx' (45.2 MB)

Export complete (2.4s)
Results saved to /home/pyrat/projects/training-notebooks
Predict:         yolo predict task=se

In [ ]:
# to export a yolov8n object detection model
ONNX_MODEL = f"{HOME}/best.onnx"
CALIBRATION_IMAGES = f"{dataset_path}/train"
# this num_classes should represent the number of classes in the model
NUM_CLASSES = 3

!hailomz compile yolov8n --ckpt={ONNX_MODEL} --hw-arch hailo8l --calib-path {CALIBRATION_IMAGES} --classes {NUM_CLASSES}



In [ ]:
# to export a yolov8s_seg segmentation model

ONNX_MODEL = f"{HOME}/best.onnx"
CALIBRATION_IMAGES = f"{dataset_path}/train"
NUM_CLASSES = 1

!echo "Y" | hailomz compile yolov8s_seg --ckpt={ONNX_MODEL} --hw-arch hailo8l --calib-path {CALIBRATION_IMAGES} --classes {NUM_CLASSES} --performance


/home/pyrat/projects/training-notebooks/.venv/bin/hailomz
[info] No GPU chosen, Selected GPU 0
[info] No GPU chosen, Selected GPU 0
E0000 00:00:1758013902.920716    9677 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1758013902.923384    9677 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
E0000 00:00:1758013902.920716    9677 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1758013902.923384    9677 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
[info] First time Hailo Dataflow Compiler is being used. Checking system requirements... (this might take a few seconds)
[info] First time Hailo Dataflow Compiler